In [73]:
# Learning Resources:
#  - https://www.pythonguis.com/tutorials/tkinter-basic-widgets/

In [74]:
import tkinter as tk
from tkinter import ttk
import yfinance as yf
import pandas as pd
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt


In [75]:
class guiWindow:
    def __init__(self):
        self.root = tk.Tk()
        self.frames={}
        self.nested_frames = {}
        self.frame_count = 0
        self.nested_frame_count = 0
        self.stock_data = None

        # Additional configuration
        self.root.title("stockr")
        self.root.minsize(255,330)
        self.root.maxsize(2550,3300)
        self.root.geometry("800x600")
        self.root.configure(background='lightgrey')


In [76]:
class guiVisuals:
    def __init__(self, guiWindow, type, target_frame=None):
        self.guiWindow = guiWindow
        self.type = type.lower()
        if target_frame is not None and target_frame in guiWindow.frames:
            self.target_frame = guiWindow.frames[target_frame]
        else:
            self.target_frame = guiWindow.root
        self.stock_data = guiWindow.stock_data
        self.dispatch_type()
    
    def dispatch_type(self):
        # Redirect type parameter to visual call
        method_name = f"create_{self.type}"
        method = getattr(self, method_name, None)
        if method:
            method()
        else:
            print(f"[Warning]: Visuals visual type '{self.type}' not yet implemented.")

    def create_label(self):
        # Create a label
        self.label = tk.Label(self.guiWindow.root, text="testing label!", font=('Helvetica', 19))
        self.label.pack()
        return self.label
        # Return to calibrate positioning!

    def create_table(self):
        if not self.stock_data:
            print("[Error]: No stock data available to create table.")
            return
        
        ticker = self.stock_data["tickers"][0]
        df = self.stock_data[f"{ticker}_data"].copy()
        df.reset_index(inplace=True)

        tree = ttk.Treeview(self.target_frame)
        tree["columns"] = list(df.columns)
        tree["show"] = "headings"

        for col in df.columns:
            tree.column(col, anchor="center")
            tree.heading(col, text=col)

        for _, row in df.iterrows():
            tree.insert("", "end", values=list(row))

        tree.pack(fill=tk.BOTH, expand=True)
        

In [77]:
class guiInteractables:
    def __init__(self, guiWindow, type):
        self.guiWindow = guiWindow
        self.type = type.lower()
        self.dispatch_type()

    def dispatch_type(self):
        # Redirect type parameter to visual call
        method_name = f"create_{self.type}"
        method = getattr(self, method_name, None)
        if method:
            method()
        else:
            print(f"[Warning]: Interactable visual type '{self.type}' not yet implemented.")
    
    def create_combobox(self):
        # Create a combobox
        self.combobox = ttk.Combobox(self.guiWindow.root, values=["Option 1", "Option 2", "Option 3"])
        self.combobox.set("Set Default Here")
        self.combobox.pack()
        return self.combobox
        # Going to want additional functionality here, calbirating size of comboboxes to max size of all boxes

    def create_entry(self):
        # Create an entry field
        self.entry = tk.Entry(self.guiWindow.root, width=30)
        self.entry.insert(0, "Enter text here")
        self.entry.pack()
        return self.entry
        # Here will want to use bind() function to display error messages if input is invalid - maybe.
    
    def create_radiobuttons(self):
        # Create radio buttons
        self.var = tk.StringVar(value="Option 1")
        self.radio1 = tk.Radiobutton(self.guiWindow.root, text="Option 1", variable=self.var, value="Option 1")
        self.radio1.pack()
        return (self.radio1)

In [78]:
class guiFrames:
    def __init__(self, guiWindow, type, width=800, height=600, parent_frame=None):
        self.guiWindow = guiWindow
        self.type = type.lower()
        self.width = width
        self.height = height
        self.parent_frame_name = parent_frame

        # Dispatch!
        self.dispatch_type()
    
    def dispatch_type(self):
        # Redirect type parameter to visual call
        method_name = f"create_{self.type}"
        method = getattr(self, method_name, None)
        if method:
            method()
        else:
            print(f"[Warning]: Frame visual type '{self.type}' not yet implemented.")

    def create_frame(self):
        # Create a frame
        self.frame = tk.Frame(self.guiWindow.root, bg='white', width=self.width, height=self.height)
        self.frame.pack()
        self.frame.pack_propagate(False)  # Prevent frame from resizing to fit contents
        # probably want to add padx and pady for spacing

        self.guiWindow.frame_count += 1
        frame_name = f"frame_{self.guiWindow.frame_count}"
        self.guiWindow.frames[frame_name] = self.frame
        self.name = frame_name # in case we want to reference this frame later

        return self.frame
    
    def create_nested_frame(self):
        # Lookup parent frame from guiWindow.frames
        if not self.parent_frame_name or self.parent_frame_name not in self.guiWindow.frames:
            print(f"[Error]: Parent frame '{self.parent_frame_name}' not found.")
            return None
        
        parent = self.guiWindow.frames[self.parent_frame_name]
        # Create a nested frame
        self.nested_frame = tk.Frame(
            parent, bg='lightblue', width=self.width, height=self.height)
        self.nested_frame.place(relx=0.5, rely=0.5, anchor='center')  # center in parent frame
        self.nested_frame.pack_propagate(False)  # Prevent frame from resizing to fit contents

        self.guiWindow.nested_frame_count += 1
        frame_name = f"nested_frame_{self.guiWindow.nested_frame_count}"
        self.guiWindow.nested_frames[frame_name] = self.nested_frame
        self.name = frame_name # in case we want to reference this frame later

        return self.nested_frame

In [79]:
class dataOperations:
    def __init__(self, tickers, start_date, end_date, interval="60m"):
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.interval = interval
        self.stock_data = {}

    def fetch_stock_data(self):
        """
        Note: plucked from old project, modified. Fetch stock data for given tickers and date range.
        
        Args:
            tickers (list): List of stock ticker symbols (e.g., ['AAPL', 'MSFT']).
            start_date (str): Start date (e.g., '2024-10-01').
            end_date (str): End date (e.g., '2024-10-31').
            interval (str): Interval for data download, recommended entries: 30m, 60m, 1d, 1wk, 1mo.
        
        Returns:
            dict: Dictionary with ticker keys and their data as values, and a list of tickers.
        """
        for ticker in self.tickers:
            # Download data for each ticker
            data = yf.download(ticker, start=self.start_date, end=self.end_date, interval=self.interval, rounding=True)
            # data = data.drop(columns=["Close", "Volume"], errors="ignore")
            self.stock_data[f"{ticker}_data"] = data
            self.stock_data["tickers"] = self.tickers
        
        guiWindow.stock_data = self.stock_data  # Update the stock_data in guiWindow
        return self.stock_data

    def correct_stock_data(self, stock_data):
        """
        Correct stock data by removing multi-index levels and renaming index column.
        
        Args:
            stock_data (dict): Dictionary of stock data with ticker keys.
        """
        for ticker_key, dataframe in stock_data.items():
            if ticker_key == "tickers":
                continue
            dataframe = dataframe.droplevel("Ticker", axis=1)
            dataframe.reset_index(inplace=True)
            dataframe.columns.name = "Data Index"
            if "Datetime" in dataframe.columns:
                dataframe["Datetime"] = pd.to_datetime(dataframe["Datetime"]).dt.strftime('%Y-%m-%d %H:%M')
            stock_data[ticker_key] = dataframe

        guiWindow.stock_data = stock_data  # Update the stock_data in guiWindow
        return stock_data

    def format_large_number(value):
        """Helper function to format large numbers for display."""
        if value is None:
            return None
        elif value >= 1e9:
            return f"{value / 1e9:.2f}B"
        elif value >= 1e6:
            return f"{value / 1e6:.2f}M"
        elif value >= 1e3:
            return f"{value / 1e3:.2f}K"
        else:
            return f"{value:.2f}"

In [85]:
# Define data request
data_op = dataOperations(
    tickers=["AAPL", "MSFT",],
    start_date="2025-06-01",
    end_date="2025-06-14",
    interval="1d")

# Fetch and assign data
window = guiWindow()
window.stock_data = data_op.fetch_stock_data()
window.stock_data = data_op.correct_stock_data(window.stock_data)

# GUI Setup
guiVisuals(window, "label")
guiInteractables(window, "combobox")
guiInteractables(window, "radiobuttons")
guiInteractables(window, "entry")
guiFrames(window, "frame", width=400, height=300)
guiFrames(window, "nested_frame", width=200, height=150, parent_frame = "frame_1")
guiVisuals(window, "table", target_frame="nested_frame_1")

# Run main loop
window.root.mainloop()

/var/folders/yt/068k7p4x3934d27j929_47ww0000gn/T/ipykernel_13475/1043090542.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=self.start_date, end=self.end_date, interval=self.interval, rounding=True)
[*********************100%***********************]  1 of 1 completed
/var/folders/yt/068k7p4x3934d27j929_47ww0000gn/T/ipykernel_13475/1043090542.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=self.start_date, end=self.end_date, interval=self.interval, rounding=True)
[*********************100%***********************]  1 of 1 completed
